In [1]:
import cv2
import numpy as np

In [2]:
# Cargar modelos pre-entrenados
face_proto = "./models/deploy.prototxt"
face_model = "./models/res10_300x300_ssd_iter_140000.caffemodel"
age_proto = "./models/age_deploy.prototxt"
age_model = "./models/age_net.caffemodel"

age_ranges = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']

face_net = cv2.dnn.readNet(face_model, face_proto)
age_net = cv2.dnn.readNet(age_model, age_proto)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0), swapRB=False, crop=False)
    face_net.setInput(blob)
    detections = face_net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Si la confianza de la detección es alta
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            face = frame[y:y1, x:x1]
            face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
            age_net.setInput(face_blob)
            age_preds = age_net.forward()
            age = age_ranges[age_preds[0].argmax()]

            label = f"Edad: {age}"
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)

            if age in ['(0-2)', '(4-6)']:
                cv2.putText(frame, "Es un bebe", (x, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow("Deteccion de bebes", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[ WARN:0@1259.850] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1259.850] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [3]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame1 = cap.read()
    if not ret:
        break

    cv2.imshow('frame', frame1)
    key = cv2.waitKey(20) & 0xFF
    if key == 27:  # Tecla ESC para salir
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np

class AgeDetector:
    def __init__(self, face_proto="./models/deploy.prototxt", face_model="./models/res10_300x300_ssd_iter_140000.caffemodel", age_proto="./models/age_deploy.prototxt", age_model="./models/age_net.caffemodel"):
        self.face_net = cv2.dnn.readNet(face_model, face_proto)
        self.age_net = cv2.dnn.readNet(age_model, age_proto)
        self.age_ranges = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
        self.model_mean_values = (78.4263377603, 87.7689143744, 114.895847746)

    def detect_age(self, frame):
        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0), swapRB=False, crop=False)
        self.face_net.setInput(blob)
        detections = self.face_net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:  # Only process detections with confidence higher than 0.5
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x1, y1) = box.astype("int")
                face = frame[y:y1, x:x1]

                face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), self.model_mean_values, swapRB=False)
                self.age_net.setInput(face_blob)
                age_preds = self.age_net.forward()
                age = self.age_ranges[age_preds[0].argmax()]

                label = f"Edad: {age}"
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)

                if age in ['(0-2)', '(4-6)']:
                    cv2.putText(frame, "Es un bebe", (x, y1 + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        return frame

def main():
    
    detector = AgeDetector(face_proto, face_model, age_proto, age_model)

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect age and display result
        frame = detector.detect_age(frame)
        cv2.imshow("Deteccion de bebes", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
